In [8]:

from math import *
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt
from CFLib.ImplCall import impVolFromFwCall
from CFLib.Heston import Heston 
from CFLib.FT_opt import ft_opt 
from CFLib.Heston import Heston 
try:
    from CIR import CIR, QT_cir_evol
except ModuleNotFoundError:
    from CFLib.CIR import CIR, QT_cir_evol
try:
    from heston_evol import __mc_heston__, heston_trj
except ModuleNotFoundError:
    from CFLib.heston_evol import __mc_heston__, heston_trj
from CFLib.euro_opt import FwEuroPut, FwEuroCall, impVolFromFwPut

#To generate heston dynamics we used the CFLib

def heston_dynamics (rand_1, rand_2, So, lmbda, eta, nubar, nu_o, rho, yrs, dt, Nt, NV, NS):
    S = heston_trj(rand_1, rand_2, So, lmbda, eta, nubar, nu_o, rho, yrs, dt, Nt, NV, NS)
    return S 

# Martingale check

def MartingaleProperty (rand1, rand2, So, lmbda, eta, nubar, nu_o, rho, yrs, dt, Nt, NV, NS) :
    S = heston_dynamics(rand1, rand2, So, lmbda, eta, nubar, nu_o, rho, yrs, dt, Nt, NV, NS)
    m = np.mean(np.mean(S, axis = 2), axis = 0)
    err = 3 * np.std(np.mean(S, axis = 2), axis = 0)/sqrt(NV)
    '''
    plt.plot(m, label = "Simulated Average")
    plt.axhline(y = 1, color = "green", linestyle= "-", label = "Price of 1")
    plt.errorbar(range(len(m)), m, yerr = err, fmt = "x", ecolor = "red", label = "Confidence Intervals")
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.ylim(0.99, 1.01)
    plt.title("Simulated Paths with Confidence Intervals")
    plt.legend()
    plt.show()
    '''
    return m, err

def MartingaleProperty_for_all_T(rand1, rand2, So, lmbda, eta, nubar, nu_o, rho, yrs, dt, NV, NS):
    for i in range (len(yrs)):
        Nt = int(yrs[i]/dt)
        m, err = MartingaleProperty(rand1, rand2, So, lmbda, eta, nubar, nu_o, rho, yrs[i], dt, Nt, NV, NS)
        x = np.linspace(0,yrs[i],Nt+1)
        plt.plot([0, yrs[i]], [1, 1], color='navy')
        plt.errorbar(x,m, yerr = err, marker="x", color="teal")
        plt.xlabel("Time")
        plt.ylabel("E[S(t)]")
        plt.show()

# different rand to not have correlated random numbers
def MC_price(rand_1,rand_2, So, lmbda, eta, nubar, nu_o, rho, yrs, dt, Nt, NV, NS,option_type,K):
    S = heston_dynamics(rand_1,rand_2, So, lmbda, eta, nubar, nu_o, rho, yrs, dt, Nt, NV, NS)
    final_prices = S[:, -1, :] 
    if option_type == "call":
        payoff = np.maximum(final_prices - K, 0)
    elif option_type == "put":
        payoff = np.maximum(K - final_prices, 0)
    else:
        raise Exception("Option type must be 'put' or 'call'")
    m = np.mean(payoff)
    std = np.std(payoff)
    err = 1.6 * std / sqrt(NV)
    return err

def price_matrix(rand_1,rand_2, So, lmbda, eta, nubar, nu_o, rho, yrs, dt, Nt, NV, NS,option_type,K):
    TK_pairs = [(t, k) for t in yrs for k in K]
    
    results = np.array([MC_price(rand_1,rand_2, So, lmbda, eta, nubar, nu_o, rho, t, float(dt*(1/t)), int(Nt*t), NV, NS,option_type,k) for t, k in TK_pairs])
    
    m_values = results[:, 0] 
    err_values = results[:, 1] 
    
    m_matrix = m_values.reshape((len(yrs), len(K)))  
    err_matrix = err_values.reshape((len(yrs), len(K))) 
    
    return m_matrix, err_matrix

def analytic_price_matrix(K, yrs, option_type, model): 
    K = np.array(K)
    yrs = np.array(yrs)
    price = np.ndarray(shape = (len(yrs), len(K)))
    for i in range(len(yrs)):
        if option_type == "put":
            price[i] = model.HestonPut(Strike = K, T = yrs[i], Xc = 10)
        elif option_type == "call":
            price[i] = model.HestonCall(Strike = K, T = yrs[i], Xc = 10)
    #price_final = pd.DataFrame(price, index=yrs, columns=K)
    return price

def difference_analytical_MC (rand_1,rand_2, So, lmbda, eta, nubar, nu_o, rho, yrs, dt, Nt, NV, NS,option_type,K, model):
    analytic = analytic_price(K, yrs, option_type, model)
    results_m, results_err = price_matrix(rand_1,rand_2, So, lmbda, eta, nubar, nu_o, rho, yrs, dt, Nt, NV, NS,option_type,K)
    final_matrix = np.ndarray((len(yrs), len(K)), dtype = bool)

    for i in range(len(T)):
        for j in range(len(K)):
            lower_bound = results_m[i, j] - results_err[i, j]
            upper_bound = results_m[i, j] + results_err[i, j]
            if lower_bound < analytic[i, j] < upper_bound:
                final_matrix[i, j] = True
            else:
                final_matrix[i, j] = False
    final_matrix = pd.DataFrame(final_matrix, index=T, columns=K)
    return final_matrix

def implied_volatility(market_price, yrs, K,type):
    if type == "call":
        implied_vol = impVolFromFwCall(market_price, yrs, K)
    if type == "put":
        implied_vol = impVolFromFwPut(market_price, yrs, K)
    return implied_vol

def implied_volatility_matrix(prices, yrs, K,type_option):
    T_array = np.array(yrs)
    K_array = np.array(K)
    impl_vol_matrix = np.empty_like(prices)
    for i, prices_row in enumerate(prices):
        Ts = T_array[i]
        if type_option == "put":
            impl_vol_matrix[i] = impVolFromFwPut(prices_row, Ts, K_array)
        if type_option == "call":
            impl_vol_matrix[i] = impVolFromFwCall(prices_row, Ts, K_array)
    result_matrix = pd.DataFrame(impl_vol_matrix, index=yrs, columns=K)
    return result_matrix


def surface_volatility(market_price,yrs,K,type_option):
    y = np.array(yrs)
    x, y = np.meshgrid(K, y)
    if type_option == "put":
        z = implied_volatility_matrix(market_price,yrs,K,"put")
    elif type_option == "call":
        z = implied_volatility_matrix(market_price,yrs,K,"call")
    else:
        raise Exception("Type must be put or call")
    volatility = np.array(z)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(x, y, volatility, cmap='viridis')  
    fig.colorbar(surf, shrink=0.5, aspect=5)
    plt.title(f"Surface Volatility - {type_option}")
    ax.set_xlabel('Moneyness')
    ax.set_ylabel('Time (years)')
    ax.set_zlabel('Implied Volatility')
    plt.show()

def effect_of_variables(rand_1,rand_2, So, lmbda, eta, nubar, nu_o, rho, yrs, dt, Nt, NV, NS,option_type,K):
    
    # Effect of lambda
    plt.figure(1)
    plt.grid()
    plt.xlabel('strike, K')
    plt.ylabel('implied volatility')
    lambdaT = [2, 3, 5, 7.7648]
    legend = []
    K = np.linspace(0.8, 1.2, 22)
    K = np.array(K).reshape([len(K),1])
    optPrice = np.zeros_like(K) 
    for lambdaTemp in lambdaT:    
        for i in range (len(K)):
            #optPrice[i] = analytic_prices(beta, sigmaTemp, T, So, K[i], type)
            optPrice[i] = MC_price(rand_1,rand_2, So, lambdaTemp, eta, nubar, nu_o, rho, yrs, dt, Nt, NV, NS,option_type,K[i])
        IV =np.zeros([len(K),1])
        for idx in range(0,len(K)):
                IV[idx] = implied_volatility(optPrice[idx], yrs, K[idx],option_type)
        plt.plot(K,IV*100.0)
        legend.append('lambda={0}'.format(lambdaTemp))
    plt.ylim([0.0,100])
    plt.legend(legend)
    plt.show()
    
    #Effect of eta 
    plt.figure(2)
    plt.grid()
    plt.xlabel('strike, K')
    plt.ylabel('implied volatility')
    etaT = [0.1, 0.5, 1.5, 2.0170]
    legend = []
    optPrice = np.zeros_like(K) 
    for etaTemp in etaT:   
       for i in range(len(K)): 
           #optPrice = analytic_prices(betaTemp, sigma, T, So, K, type)
            optPrice[i] = MC_price(rand_1,rand_2, So, lmbda, etaTemp, nubar, nu_o, rho, yrs, dt, Nt, NV, NS,option_type,K[i])

       IV =np.zeros([len(K),1])
       for idx in range(0,len(K)):
            IV[idx] = implied_volatility(optPrice[idx], yrs, K[idx], option_type)
       plt.plot(K,IV*100.0)
       legend.append('eta={0}'.format(etaTemp))
    plt.legend(legend)
    plt.show()
